# T1071.003 Mail Protocols

-----------------------------------------------------------------------

## Technique Description

Adversaries may communicate using application layer protocols associated with electronic mail delivery to avoid detection/network filtering by blending in with existing traffic. Commands to the remote system, and often the results of those commands, will be embedded within the protocol traffic between the client and server. 

Protocols such as SMTP/S, POP3/S, and IMAP that carry electronic mail may be very common in environments.  Packets produced from these protocols may have many fields and headers in which data can be concealed. Data could also be concealed within the email messages themselves. An adversary may abuse these protocols to communicate with systems under their control within a victim network while also mimicking normal, expected traffic. 

## Technique Detection

Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect application layer protocols that do not follow the expected protocol standards regarding syntax, structure, or any other variable adversaries could leverage to conceal data.(Citation: University of Birmingham C2)

-----------------------------------------------------------------------

### Tactics:

  *   Command-And-Control

### Platforms:

  * Linux

  * macOS

  * Windows

### Data Sources:

  * **Network Traffic:** Network Traffic Flow

  * **Network Traffic:** Network Traffic Content

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
| Kimsuky | [Kimsuky](https://attack.mitre.org/groups/G0094) has used e-mail to send exfiltrated data to C2 servers.(Citation: CISA AA20-301A Kimsuky)| 
| SilverTerrier | [SilverTerrier](https://attack.mitre.org/groups/G0083) uses SMTP for C2 communications.(Citation: Unit42 SilverTerrier 2018)| 
| APT32 | [APT32](https://attack.mitre.org/groups/G0050) has used email for C2 via an Office macro.(Citation: Cybereason Oceanlotus May 2017)(Citation: Cybereason Cobalt Kitty 2017)| 
| Turla | [Turla](https://attack.mitre.org/groups/G0010) has used multiple backdoors which communicate with a C2 server via email attachments.(Citation: Crowdstrike GTR2020 Mar 2020)| 
| APT28 | [APT28](https://attack.mitre.org/groups/G0007) has used IMAP, POP3, and SMTP for a communication channel in various implants, including using self-registered Google Mail accounts and later compromised email servers of its victims.(Citation: FireEye APT28)(Citation: Cybersecurity Advisory GRU Brute Force Campaign July 2021)| 
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1071/003)

  * [University Of Birmingham C2](https://arxiv.org/ftp/arxiv/papers/1408/1408.1136.pdf), Gardiner, J.,  Cova, M., Nagaraja, S. (2014, February). Command & Control Understanding, Denying and Detecting. Retrieved April 20, 2016.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 30 June 2022

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres, CTR Servando Quinones, SSgt Sengsouriya Kapkeo

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Adversaries may use mail traffic from compromised accounts to conduct C2 by beaconing or downloading 2nd stage malware while hiding in legitimate network traffic. APT 28 will attempt to utilize smtp to contact C2 servers as a means of living of the land once they have compromised email accounts. APT 28 may attempt to retrieve commands through the use of attachments, these will probably range from very small files to execute specific commands,screenshots, and data exfiltration for files of interest. 

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| APT28, Turla| 1 | 1, 2|


## Detection Blindspots

- Web-mail typically is encrypted leaving us in the dark about what is going on in the traffic. Even if you are able to capture and see the contents depending on the size of the network this could be a large data set. 

- Network would only be able to see the SMTP traffic, not POP3 or IMAP

## Analytical References

  * [Report-APT28](https://www.fireeye.com/content/dam/fireeye-www/global/en/current-threats/pdfs/rpt-apt28.pdf)
  * [ESET Sednit Part 2](https://www.welivesecurity.com/wp-content/uploads/2016/10/eset-sednit-part-2.pdf)
  * [Turla Crowdstrike](https://go.crowdstrike.com/rs/281-OBQ-266/images/Report2020CrowdStrikeGlobalThreatReport.pdf)

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** Chopstick/Xagent uses a proxy server to relay traffic between infected computers and a C2 server. The source code of the proxy server may be left in an open directory on the proxy server, which may be indexed by the Google search engine.

  * **Source:** Sysmon

  * **Tool:** Kibana

  * **Notes:** The proxy code is a set of Python scripts containing more than 12,200 lines of code among 14 files.

  * **Query:** ```Event_ID: 1 AND *.py*```


-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

-   APT28 used SMTP as a communication channel in various implants, initially using self-registered Google Mail accounts and later compromised email servers of its victims.
    Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server).
    Analyze packet contents to detect application layer protocols that do not follow the expected protocol standards regarding syntax, structure, or any other variable adversaries could leverage to conceal data.
    When sending data to the C2 server, it is sent using SMTP over TLS. The message is retrieved by the proxy server and forwarded to the C2 server.
    XAgent when receiving data from the C2 server, it is sent using POP3 over TLS. The email address to receive messages will be different than the one used to send messages. (Xagent version 2 Linux channels)
    The MailChannel object is an implementation of Xagent communication channel over emails, where messages are sent and received as attachments to emails.

#### Analytic 1

  * **Information:** Chopstick/Xagent uses a "MailChannel" object for communications over emails by sending and receiving messages as attachments. Look for communication between users that only use attachments.

  * **Source:** Network Traffic, PCAP

  * **Tool:** Arkime, Kibana

  * **Notes:** This hung will involving isolating traffic of interest and correlating with host logs to get a better understanding of the network connections involving client to server email. Isolating emails with attachements may assist in narrowing the traffic and allow for closer examination of files being sent and received.

  * **Query Arkime:** ```protocols == [smtp, imap, pop3] && email.md5 == EXISTS!```
  * **Query Kibana:** ```protocol: (smtp or imap or pop3) AND email.md5: *```

#### Analytic 2

  * **Information:** The email body and the attachment name are the base64 encodings of pseudo-randomly generated values. Look for communication between users that only use attachments.

  * **Source:** Network Traffic, PCAP

  * **Tool:** Arkime, Kibana

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query Arkime:** ```email.bodymagic == EXISTS! && email.fn == EXISTS!```
  * **Query Kibana:** ```email.bodymagic: * AND email.fn: *```

#### Analytic 3

  * **Information:** Identify email leving the MP network

  * **Source:** Network Traffic, PCAP

  * **Tool:** Kibana

  * **Notes:** This will have alot of network traffic, collaborate with host for possible suspicious or authorized external email providers.

  * **Query Kibana:** ```protocol:smtp or dstPort:(25 or 465 or 587) AND srcIp:(10.0.0.0/8 or 172.16.0.0/12 or 192.168.0.0/16) AND NOT dstIp:(10.0.0.0/8 or 127.0.0.0/8 or 172.16.0.0/12 or 192.168.0.0/16)```

* **Deep Dive**
- Arkime options:
  - Mail Protocols: smtp, imap, pop3
    - Ask the mission partner what they use for email
    - Keep in mind that some printers use smtp when scanning documents.


- <font color="purple"> Subject: </font> (email.subject) Once you have a subject that you find interesting this could give you all of the potentially compromised accounts that are contacting separate C2 servers using this technique.

- <font color="purple"> Senders: </font> (email.src) This is the sender of the email it is important too note to identify a potential IOC.

- <font color="purple"> Destination: </font> (email.dst) This is the receiver of the email will identify what accounts are known (not necessarily compromised) by the adversary.

- <font color="red"> Filenames: </font> Look for file extensions that could potentially contain malicious scripts or malware then search for that file being transferred to other accounts or to other boxes.

<p align="center">
<img src="../../Images/T1071.003_Mail_Protocols1.png">
</p>

* **If you are unable to find a starting point it might be worth filtering out if Arkime has detected an application**
  
  -     Email.bodymagic == application*


 In the first example Arkime has detected a "/zip" which turns out to be an obfuscated base 64 encoded docx file

- This is something that should definitely be looked into further and given to the host team so that they can identify the file on that particular machine.

<p align="center">
<img src="../../Images/T1071.003_Mail_Protocols2.png">
</p>

Below is a screen shot of the transfer Arkime was able to identify the <font color="green">filename and base64 </font> encoding however is not able to assemble the file if the operator tries to use the "Show Image & files" button.

  - This is most likely due to the padding or chunking  the data with a series of "A"

<p align="center">
<img src="../../Images/T1071.003_Mail_Protocols3.png">
</p>


- In the second example  Arkime has detected a <font color="red">"/vnd.ms-tnef"</font>

  - "vnd" is a content-type header that identifies it as Vendor specific
  - ".ms" most likely is identifying Microsoft as the vendor
  - "-tnef" Transport neutral Encapsulation Format is the Microsoft-specific format for encapsulating MAPI message properties. A TNEF message contains a plain text version of the message and an attachment that packages the original formatted version of the message.
    - Can contain embedded pictures or embedded Office documents
    - Outlook and other email clients that fully understand the TNEF process of Winmail.dat attachment can display the original message content without ever displaying the .dat attachment.

<p align="center">
<img src="../../Images/T1071.003_Mail_Protocols4.png">
</p>

- Below is a screen shot of the transfer Arkime was able to identify the <font color="green">Filename</font> and when you hit "Show Image & Files" you can see the <font color="blue">winmail.dat</font> these can be benign however they often contain information about the host and are of interest to an adversary looking to know what they have exploited and possibly how they can begin to move around. 

<p align="center">
<img src="../../Images/T1071.003_Mail_Protocols5.png">
</p>


